In [1]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time

def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=0)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((4,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 53998
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 53999
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54000
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54001
INFO:root:(('127.0.0.1', 53998)) Conexión aceptada con ('127.0.0.1', 53999)
INFO:root:(('127.0.0.1', 53999)) Conexión aceptada con ('127.0.0.1', 54000)
INFO:root:(('127.0.0.1', 54000)) Conexión aceptada con ('127.0.0.1', 53998)
INFO:root:(('127.0.0.1', 54000)) Conexión aceptada con ('127.0.0.1', 54001)
INFO:root:(('127.0.0.1', 54001)) Conexión aceptada con ('127.0.0.1', 53999)
INFO:root:(('127.0.0.1', 54001)) Conexión aceptada con ('127.0.0.1', 53998)
INFO:root:(('127.0.0.1', 53998)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 53999)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 53998)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 54000)) Init

In [3]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

for thread in threading.enumerate(): 
    print(thread.name)
    
print("\n\n")
    
nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    nodes[1].stop()
    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
test_node_down_on_learning(4)
print("\n\n\n\n")
time.sleep(3)
for thread in threading.enumerate(): 
    print(thread.name)
    

MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2



Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54251
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54252
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54253
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 54254
INFO:root:(('127.0.0.1', 54251)) Conexión aceptada con ('127.0.0.1', 54252)
INFO:root:(('127.0.0.1', 54252)) Conexión aceptada con ('127.0.0.1', 54253)
INFO:root:(('127.0.0.1', 54253)) Conexión aceptada con ('127.0.0.1', 54251)
INFO:root:(('127.0.0.1', 54253)) Conexión aceptada con ('127.0.0.1', 54254)
INFO:root:(('127.0.0.1', 54254)) Conexión aceptada con ('127.0.0.1', 54252)
INFO:root:(('127.0.0.1', 54254)) Conexión aceptada con ('127.0.0.1', 54251)
INFO:root:(('127.0.0.1', 54251)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 54252)) Initialicing Model Weights
INFO:roo

In [2]:
#import threading
for thread in threading.enumerate(): 
    print(thread.name)

MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2


In [6]:
import sys

def get_thread_position(thread):
    frame = sys._current_frames().get(thread.ident, None)
    if frame:
        return frame.f_code.co_filename, frame.f_code.co_name, frame.f_code.co_firstlineno

get_thread_position(threading.enumerate()[-1])

('/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/node.py',
 '__on_round_finished',
 260)

In [1]:
from p2pfl.const import HEARTBEAT_FREC, SOCKET_TIEMOUT
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
from test.fixtures import two_nodes, four_nodes


import threading
for thread in threading.enumerate(): 
    print(thread.name)
    
n1 = Node(MLP(),MnistFederatedDM())
n2 = Node(MLP(),MnistFederatedDM())
print("\n")
def test_bad_binary_model():

    n1.start()
    n2.start()

    n1.connect_to(n2.host,n2.port)
    time.sleep(0.1) 

    # Start Learning
    n1.set_start_learning(rounds=2,epochs=0)
 
    # Adding noise to the buffer
    for _ in range(200):
        n1.neightboors[0].param_bufffer += "noise".encode("utf-8")
    
    while not n1.round is None and not n2.round is None:
        time.sleep(0.1)
        
    n1.stop()
    n2.stop()
    
test_bad_binary_model()

time.sleep(10)

for thread in threading.enumerate(): 
    print(thread.name)
    
print("\n")



MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2
Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000


INFO:root:Nodo a la escucha en 127.0.0.1 54998
INFO:root:Nodo a la escucha en 127.0.0.1 54999
INFO:root:(('127.0.0.1', 54999)) Conexión aceptada con ('127.0.0.1', 54998)
INFO:root:(('127.0.0.1', 54998)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 54998)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 54999)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54998)) Broadcasting model to 1 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 54998)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1', 54998)) Training...
INFO:root:(('127.0.0.1', 54999)) Model initialized
INFO:root:(('127.0.0.1', 54998)) Model added (1/2) from ('127.0.0.1', 54998)
INFO:root:(('127.0.0.1', 54998)) Broadcasting model to 1 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 54999)) Model added (1/2) from ('127.0.0.1', 54

In [4]:

for thread in threading.enumerate(): 
    print(thread.name)
    

MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2


In [6]:
print(n1.round)

None
